In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, Input
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

import numpy as np

from tensorflow.keras.optimizers import Adam

import numpy as np

from sklearn.metrics import classification_report, confusion_matrix

In [2]:
def preprocess_input_grayscale(img):
    return np.repeat(img, 3, axis=-1)

In [3]:
train_datagen = ImageDataGenerator(
    # preprocessing_function=preprocess_input_grayscale,
    rescale=1./255,
    rotation_range=20,
    horizontal_flip=True,
    shear_range=0.2,
    fill_mode='wrap',
)

train_generator = train_datagen.flow_from_directory(
    'Dataset_train',
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
)

val_datagen = ImageDataGenerator(
    # preprocessing_function=preprocess_input_grayscale,
    rescale=1./255
)

val_generator = val_datagen.flow_from_directory(
    'Dataset_testing',
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical'
)

Found 4444 images belonging to 8 classes.
Found 1114 images belonging to 8 classes.


In [4]:
num_classes = len(train_generator.class_indices)

model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(224,224,3)),
    tf.keras.layers.Reshape((224, 224 * 3)),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(64),
    # tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax') #Output
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', 'precision', 'recall'])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape (Reshape)               │ (None, 224, 672)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 224, 128)       │       410,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │        66,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 8)              │         2,056 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,184,264 (4.52 MB)

 Trainable params: 1,184,264 (4.52 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=15,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    verbose=1
)

c:\Users\rizfi\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
139/139 ━━━━━━━━━━━━━━━━━━━━ 78s 529ms/step - accuracy: 0.2367 - loss: 1.9915 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.2531 - val_loss: 1.9816 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/15
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 692us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 3/15


c:\Users\rizfi\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


139/139 ━━━━━━━━━━━━━━━━━━━━ 58s 417ms/step - accuracy: 0.2487 - loss: 1.9763 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.2540 - val_loss: 1.9503 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/15
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 223us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 5/15
139/139 ━━━━━━━━━━━━━━━━━━━━ 66s 471ms/step - accuracy: 0.2473 - loss: 1.9642 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.2567 - val_loss: 1.9504 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 6/15
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 237us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 7/15
139/139 ━━━━━━━━━━━━━━━━━━━━ 63s 455ms/step - accuracy: 0.2572 - loss: 1.9633 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.2478 - val_loss: 1.9572 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 8/15
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s

In [6]:
# Mendapatkan nilai akurasi untuk setiap epoch dari history
train_accuracy = history.history['accuracy']  # Akurasi pada data training
val_accuracy = history.history['val_accuracy']  # Akurasi pada data validasi

# Mencetak akurasi di setiap epoch
print("Training Accuracy per Epoch:")
for epoch, acc in enumerate(train_accuracy, 1):
    print(f"Epoch {epoch}: {acc:.4f}")

print("\nValidation Accuracy per Epoch:")
for epoch, val_acc in enumerate(val_accuracy, 1):
    print(f"Epoch {epoch}: {val_acc:.4f}")

Training Accuracy per Epoch:
Epoch 1: 0.2441
Epoch 2: 0.0000
Epoch 3: 0.2532
Epoch 4: 0.0000
Epoch 5: 0.2525
Epoch 6: 0.0000
Epoch 7: 0.2563
Epoch 8: 0.0000
Epoch 9: 0.2554
Epoch 10: 0.0000
Epoch 11: 0.2577
Epoch 12: 0.0000
Epoch 13: 0.2610
Epoch 14: 0.0000
Epoch 15: 0.2563

Validation Accuracy per Epoch:
Epoch 1: 0.2531
Epoch 2: 0.2540
Epoch 3: 0.2567
Epoch 4: 0.2478
Epoch 5: 0.2585
Epoch 6: 0.2567
Epoch 7: 0.2612
Epoch 8: 0.2496


In [7]:

model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy', 'precision', 'recall'])

fine_tune_history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=20,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    verbose=1
)

Epoch 1/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 69s 457ms/step - accuracy: 0.2631 - loss: 1.9503 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.2522 - val_loss: 1.9431 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 518us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 3/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 65s 469ms/step - accuracy: 0.2651 - loss: 1.9419 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.2531 - val_loss: 1.9409 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 244us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 5/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 64s 461ms/step - accuracy: 0.2550 - loss: 1.9403 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.2531 - val_loss: 1.9397 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 6/20
139/139 ━━━━━━━━━━━━

In [8]:
# Mendapatkan nilai akurasi untuk setiap epoch dari history
train_accuracy = fine_tune_history.history['accuracy']  # Akurasi pada data training
val_accuracy = fine_tune_history.history['val_accuracy']  # Akurasi pada data validasi
# Mencetak akurasi di setiap epoch
print("Training Accuracy per Epoch:")
for epoch, acc in enumerate(train_accuracy, 1):
    print(f"Epoch {epoch}: {acc:.4f}")

print("\nValidation Accuracy per Epoch:")
for epoch, val_acc in enumerate(val_accuracy, 1):
    print(f"Epoch {epoch}: {val_acc:.4f}")

Training Accuracy per Epoch:
Epoch 1: 0.2581
Epoch 2: 0.0000
Epoch 3: 0.2601
Epoch 4: 0.0000
Epoch 5: 0.2592
Epoch 6: 0.0000
Epoch 7: 0.2592
Epoch 8: 0.0000
Epoch 9: 0.2590
Epoch 10: 0.0000
Epoch 11: 0.2565
Epoch 12: 0.0000
Epoch 13: 0.2563
Epoch 14: 0.0000
Epoch 15: 0.2559
Epoch 16: 0.0000
Epoch 17: 0.2577
Epoch 18: 0.0000
Epoch 19: 0.2597
Epoch 20: 0.0000

Validation Accuracy per Epoch:
Epoch 1: 0.2522
Epoch 2: 0.2531
Epoch 3: 0.2531
Epoch 4: 0.2531
Epoch 5: 0.2540
Epoch 6: 0.2558
Epoch 7: 0.2549
Epoch 8: 0.2549
Epoch 9: 0.2558
Epoch 10: 0.2585


In [9]:
test_images, test_labels = next(val_generator)

test_images = test_images[:10]
test_labels = test_labels[:10]

predictions = model.predict(test_images)

predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(test_labels, axis=1)

# Tampilkan hasil prediksi untuk 10 data
for i in range(10):
    print(f"Predicted: {predicted_classes[i]}, True: {true_classes[i]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 361ms/step
Predicted: 4, True: 6
Predicted: 6, True: 1
Predicted: 4, True: 6
Predicted: 4, True: 4
Predicted: 4, True: 7
Predicted: 4, True: 5
Predicted: 4, True: 7
Predicted: 4, True: 4
Predicted: 4, True: 6
Predicted: 6, True: 4


In [ ]:
y_pred = model.predict(val_generator)
y_pred_classes = np.argmax(y_pred, axis=1)

# Get true labels from the test generator
y_true = val_generator.classes  # True class labels

# Get class names from the generator
class_names = list(val_generator.class_indices.keys())

# Generate the classification report
report = classification_report(y_true, y_pred_classes, target_names=class_names)
print("Classification Report: \n", report)

# Optional: Confusion matrix for additional insights
conf_matrix = confusion_matrix(y_true, y_pred_classes)
print("Confusion Matrix:\n", conf_matrix)

35/35 ━━━━━━━━━━━━━━━━━━━━ 4s 92ms/step
Classification Report:
               precision    recall  f1-score   support

       anger       0.00      0.00      0.00       178
    contempt       0.00      0.00      0.00        42
     disgust       0.00      0.00      0.00        88
        fear       0.00      0.00      0.00       114
   happiness       0.25      0.82      0.38       282
  neutrality       0.00      0.00      0.00       105
     sadness       0.15      0.18      0.17       150
    surprise       0.00      0.00      0.00       155

    accuracy                           0.23      1114
   macro avg       0.05      0.12      0.07      1114
weighted avg       0.08      0.23      0.12      1114

Confusion Matrix:
 [[  0   0   0   0 149   0  29   0]
 [  0   0   0   0  37   0   5   0]
 [  0   0   0   0  74   0  14   0]
 [  0   0   0   0  99   0  15   0]
 [  0   0   0   0 231   0  51   0]
 [  0   0   0   0  87   0  18   0]
 [  0   0   0   0 123   0  27   0]
 [  0   0   0   0 139

c:\Users\rizfi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\rizfi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\rizfi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo